In [ ]:
# from langchain_ollama import ChatOllama

# mistral = ChatOllama(
#     model="mistral",
#     temperature=0,
# )

In [ ]:
# from langchain_ollama import OllamaEmbeddings

# embeddings = OllamaEmbeddings(
#     model="mistral",
# )

###  Using Groqcloud API

In [20]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("GROQ_API_KEY")

In [72]:
from langchain_groq import ChatGroq

groqLLM = ChatGroq(
    model="deepseek-r1-distill-qwen-32b", 
    api_key=api_key,
    temperature=0.7, 
    max_tokens=512    # Limit response length
)

In [73]:
from langchain_huggingface import HuggingFaceEmbeddings
import tqdm as notebook_tqdm

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [31]:
result = embeddings.embed_query("hello")

In [32]:
len(result)

384

In [47]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [49]:
import time

index_name = "personal" 

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
# index.delete(delete_all=True)

{}

In [50]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [29]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader

def custom_loader(file_path: str):
    if file_path.endswith(".pdf"):
        return PyPDFLoader(file_path)
    elif file_path.endswith(".txt"):
        return TextLoader(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_path}")

loader = DirectoryLoader("personal", glob="**/*", show_progress=True, loader_cls=custom_loader)
docs = loader.load()

 86%|████████▌ | 6/7 [00:00<00:00,  7.35it/s]


In [52]:
# vector_store.add_documents(docs)

In [67]:
groq_retreiver = vector_store.as_retriever(search_kwargs={'k':2})

In [74]:
from langchain_core.tools import tool

@tool
def marks_reader(self, file_path:str) -> str:
    """Read content from a text file."""
    try:
        if not os.path.exists(file_path):
            return f"Error: File not found at path {file_path}"
        
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        return content
    except Exception as e:
        return f"Error reading file: {str(e)}"
    
tools = [marks_reader]
groqLLM_with_tools = groqLLM.bind_tools(tools)

In [80]:
from langchain.prompts import PromptTemplate

template = """
    You are a digital copy of this user that can provide information based his documents on both vector search from a database and other tools.

    ### Task:
    You will be provided with a user query. Your goal is to respond to the query by doing the following:
    1. Retrieve relevant information from the vector database (Pinecone) that best matches the query.
    2. If additional context or information is required that can be retrieved from tools, use the appropriate tool.
    3. Combine the information from the database and tools to provide a well-structured and complete response.

    ### User Query:
    {user_query}

    ### Relevant Documents (from Pinecone vector database):
    {retrieved_documents}

    ### Tool Usage:
    If the relevant documents or context are not sufficient to answer the query, use the tools you have. For example, if the user is asking for marks of certain subjects, use the marks_reader tool to extract information from the marks.txt file.
    If you used any tools, describe how the tool was used in your response.

    ### Answer:
    Your answer should be concise, clear, and comprehensive, using the retrieved documents and any tool-assisted information.
 """

prompt = PromptTemplate(
    input_variables=["user_query", "retrieved_documents"],
    template=template,
)

In [106]:
chain = prompt | groqLLM_with_tools

In [108]:
chain.invoke({"user_query": "Hello how are you",
              "retrieved_documents": "there are no documents"})

AIMessage(content="Hello! I'm doing well, thank you for asking. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 370, 'total_tokens': 394, 'completion_time': 0.171428571, 'prompt_time': 0.021585505, 'queue_time': 0.053641697, 'total_time': 0.193014076}, 'model_name': 'deepseek-r1-distill-qwen-32b', 'system_fingerprint': 'fp_d458a8aba5', 'finish_reason': 'stop', 'logprobs': None}, id='run-f641d736-2b41-4b33-98b7-7ce44146db46-0', usage_metadata={'input_tokens': 370, 'output_tokens': 24, 'total_tokens': 394})

In [ ]:
# user_query = input("Your query :: \n")
# retrieved_documents = mistral_retriever.invoke(user_query)

# chain = prompt | mistral_with_tools

# response = chain.invoke({"user_query": user_query, "retrieved_documents": retrieved_documents})
# print(response.content)

In [ ]:
# print(response.content)

 The name of the candidate mentioned in the provided document is Nabaraj Subedi.


### Let's add memory to the chatbot using **langgraph**

In [101]:
from typing import TypedDict, Annotated, Sequence
from langgraph.graph import START, END, StateGraph
import operator
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

class GraphState(TypedDict):
    user_query : str
    retrieved_documents : Sequence[str]
    messages: Annotated[list, add_messages]


def input_node(state: GraphState) -> GraphState:
    if not state.get("messages"):
        state["messages"] = [HumanMessage(content=state["user_query"])]
    return state

def retrieval_node(state: GraphState)->GraphState:
    retrieved_documents = groq_retreiver.invoke(state['user_query'])
    state['retrieved_documents'] = retrieved_documents
    return state

def processing_node(state: GraphState)->GraphState:
    chain = prompt | groqLLM_with_tools
    response = chain.invoke(
        {
            "messages": state["messages"],
            "user_query": state['user_query'],
            "retrieved_documents": state['retrieved_documents']
        }
    )
    print(f"reponse of the processing node {response}")
    state["messages"] = state["messages"] + [response]
    return state

tool_node = ToolNode(tools=tools)

workflow = StateGraph(GraphState)

workflow.add_node("input", input_node)
workflow.add_node("retrieval", retrieval_node)
workflow.add_node("processing", processing_node)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "input")
workflow.add_edge("input","retrieval")
workflow.add_edge("retrieval","processing")

workflow.add_conditional_edges("processing", tools_condition)  # if tool call is there, it is called
workflow.add_edge("tools", "processing")  # if toolcall is made then tool execution should occur, if occur, this is ..

workflow.add_edge("processing",END)

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

app = workflow.compile(checkpointer=memory)

In [103]:
user_query = input("You:: \n")
initial_state = {
    "user_query": user_query,
    "retrieved_documents": [],
    "messages": []
}
config = {"configurable": {"thread_id": "1"}}

result = app.invoke(initial_state,config=config)
print(result["messages"][-1].content)

reponse of the processing node content="The user's name is **NIRAJAN PAUDEL**, as clearly stated in the resume document. There's no need to use the `marks_reader` tool since the name is already provided in the available documents." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 194, 'prompt_tokens': 1054, 'total_tokens': 1248, 'completion_time': 1.385714286, 'prompt_time': 0.063373553, 'queue_time': 0.053424325999999994, 'total_time': 1.449087839}, 'model_name': 'deepseek-r1-distill-qwen-32b', 'system_fingerprint': 'fp_0852292947', 'finish_reason': 'stop', 'logprobs': None} id='run-c9c24764-8485-4207-8fd7-d0206cacdb10-0' usage_metadata={'input_tokens': 1054, 'output_tokens': 194, 'total_tokens': 1248}
The user's name is **NIRAJAN PAUDEL**, as clearly stated in the resume document. There's no need to use the `marks_reader` tool since the name is already provided in the available documents.


In [100]:
result

{'user_query': 'what are my marks in C programming?',
 'retrieved_documents': [Document(id='c955d3f5-2d9c-4cd6-90d8-f4f5017310b0', metadata={'page': 0.0, 'source': 'personal/IELTS result.pdf'}, page_content='Centre Number\nSex (M/F)\nDate Test Report \nForm Number\nCEFR \nLevel\nDate\nTest Results\nValidation stamp\nIELTs· \n□ □ \nee BRITISH \n••coUNCIL \n□ \nJ..idp \n□ \nL □ \n,,,,f,. Cambridge Assessment \n::: English \n□ \nScheme Code\nCandidate Details\nFamily Name\nFirst Name(s)\nCandidate ID\nOverall\nBand\n Score\nRecognising \norganisations must \nverify this score at \nielts.org/verify\nAdministrator Comments\nReading Writing SpeakingListening\nDate of Birth\nCountry or \nRegion of Origin\nCountry of \nNationality\nFirst Language\nAdmission to undergraduate and post graduate courses should be based on the ACADEMIC Reading and Writing Modules. \nGENERAL TRAINING Reading and Writing Modules are not designed to test the full range of language skills required for academic purposes